# Worldwide Earthquake Events API - Gold Layer Processing

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
# ensure the below library is installed on your fabric environment
import reverse_geocode as rg

StatementMeta(, d7c00461-cbee-4484-817b-ab571c19c319, 5, Finished, Available)

In [2]:
df = spark.read.table('earthquake_events_silver')

StatementMeta(, 73f93af4-914b-437a-9a78-5964179f79a1, 6, Finished, Available)

In [4]:
def get_country_code_details(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_code_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = ((float(lat), float(lon)),(float(lat), float(lon)))
    return rg.search(coordinates)[0].get('country_code')

def get_city_details(lat, lon):
    """
    Retrieve the city name for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: City name of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_city_details(34.052235, -118.243683)
    'Los Angeles'
    """
    coordinates = ((float(lat), float(lon)),(float(lat), float(lon)))
    return rg.search(coordinates)[0].get('city')

StatementMeta(, d7c00461-cbee-4484-817b-ab571c19c319, 8, Finished, Available)

In [5]:
# registering the udfs
get_country_code_details_udf = udf(get_country_code_details, StringType())
get_city_details_udf = udf(get_city_details, StringType())

StatementMeta(, d7c00461-cbee-4484-817b-ab571c19c319, 9, Finished, Available)

In [6]:
# adding country_code and city attributes
df_with_location = df.withColumn("country_code", get_country_code_details_udf(df["latitude"], df["longitude"])).withColumn("city", get_city_details_udf(df["latitude"], df["longitude"]))

StatementMeta(, d7c00461-cbee-4484-817b-ab571c19c319, 10, Finished, Available)

In [8]:
# appending the data to the gold table
df_with_location.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, d7c00461-cbee-4484-817b-ab571c19c319, 12, Finished, Available)